In [2]:
from openmm import unit
from openmmtools import testsystems, states, mcmc
from openmmtools.multistate import ParallelTemperingSampler, MultiStateReporter
import tempfile
import os

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [16]:
from openmm.app import *

pdb = PDBFile('/home/jakub/phd/openmm-md/data/241010_FoldingUponBinding/output/SUMO-1C/241128-MetaD/sumo1c_equilibrated.pdb')

forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

system = forcefield.createSystem(
    topology=pdb.topology, 
    nonbondedMethod=PME,
    nonbondedCutoff=1*unit.nanometer,
    constraints=HBonds
    )

In [17]:
n_replicas = 4  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.


# thermodynamic state holds information on ensemble parameters like temperture and pressure
reference_state = states.ThermodynamicState(system=system, temperature=T_min)


In [18]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)

In [19]:
storage_path = 'replica_exchange.nc'
reporter = MultiStateReporter(storage_path, checkpoint_interval=10)

In [31]:
# Create sampler state with both positions and box vectors
sampler_state = states.SamplerState(
    positions=pdb.positions,
    box_vectors=system.getDefaultPeriodicBoxVectors()
)

simulation = ParallelTemperingSampler(
    mcmc_moves=move, 
    number_of_iterations=2
    )

simulation.create(
    thermodynamic_state=reference_state,
    sampler_states=[sampler_state],
    storage=reporter, 
    min_temperature=T_min,
    max_temperature=T_max, 
    n_temperatures=n_replicas)


simulation.run()


RuntimeError: Storage file replica_exchange.nc already exists; cowardly refusing to overwrite.

In [29]:
simulation.read_status('replica_exchange.nc')

Status(iteration=2, target_error=None, is_completed=True)

In [30]:
from openmmtools.multistate import ParallelTemperingAnalyzer

analyzer = ParallelTemperingAnalyzer(storage_path)


/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py:960: SyntaxWarning: invalid escape sequence '\l'
  """
/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py:995: SyntaxWarning: invalid escape sequence '\s'
  """
/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py:1415: SyntaxWarning: invalid escape sequence '\l'
  """
/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py:1480: SyntaxWarning: invalid escape sequence '\s'
  """Return an MBAR-ready decorrelated energy matrix.


ValueError: reporter must be a MultiStateReporter instance